In [1]:
import yfinance as yf
import pandas as pd
import plotly.express as px

In [2]:
df = yf.download("TCS.NS", period="6mo", multi_level_index=False)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
df

,Close,High,Low,Open,Volume
Date,,,,,
2024-11-18,3956.431152,4146.894998,3927.590842,4146.894998,3313944
2024-11-19,3976.166504,4067.313397,3951.115881,3956.824683,1776233
2024-11-21,4008.944092,4028.236409,3960.860488,3981.432456,2384347
2024-11-22,4177.999023,4188.186720,3990.389589,4007.122846,3096553
2024-11-25,4247.393066,4260.189087,4185.480109,4212.843715,4052167
...,...,...,...,...,...
2025-05-12,3620.399902,3630.500000,3502.199951,3505.000000,2228678
2025-05-13,3515.000000,3623.000000,3510.500000,3620.399902,3062887
2025-05-14,3548.699951,3561.000000,3508.600098,3521.000000,2119499
